In [1]:
import numpy as np 
import pandas as pd 

countries_report = pd.read_csv("countries.csv", header = 0, low_memory=False)
happiness_report = pd.read_csv("2017.csv", header = 0, low_memory=False)

happiness_report = happiness_report.sample(frac=1)
print happiness_report.shape
print countries_report.shape

(151, 12)
(227, 20)


In [2]:
happy_data = []
country_data = []

rows = happiness_report.shape[0]
for i in range(rows):
    key = unicode("Country", "utf-8")
    c = happiness_report.loc[happiness_report.index[[i]], key]
    c_data = happiness_report.loc[happiness_report.index[[i]]]
#     print c_data.values[0]
    happy_data.append(c_data.values[0])
    
    # Gets corresponding countries.csv data for each country in 2017.ccsv
    Country = happiness_report.loc[happiness_report.index[[i]], key].values
#     print Country[0]
    
    col = countries_report['Country']
    index = -1
    for i in col:
        index += 1
        i = str(i)
        if Country[0] in i:
#             print i
            idx = index
            break
        
    data = countries_report.loc[countries_report.index[idx]]
    country_data.append(data)

In [3]:
print len(happy_data)
print len(country_data)

#sanity check
idx = 0
for i in country_data:
    print happy_data[idx][0],
    print i[0]
    idx += 1

151
151
Iceland Iceland 
Moldova Moldova 
Macedonia Macedonia 
Ethiopia Ethiopia 
Azerbaijan Azerbaijan 
Portugal Portugal 
Russia Russia 
Sierra Leone Sierra Leone 
Mali Mali 
Netherlands Netherlands 
Cambodia Cambodia 
Ecuador Ecuador 
Norway Norway 
Slovakia Slovakia 
Bolivia Bolivia 
Cyprus Cyprus 
Hungary Hungary 
Brazil Brazil 
South Africa South Africa 
Sweden Sweden 
Zambia Zambia 
Jamaica Jamaica 
Philippines Philippines 
Libya Libya 
Chad Chad 
Somalia Somalia 
Thailand Thailand 
Taiwan Taiwan 
Algeria Algeria 
Bulgaria Bulgaria 
Ireland Ireland 
Senegal Senegal 
Slovenia Slovenia 
Estonia Estonia 
Egypt Egypt 
Haiti Haiti 
France France 
Panama Panama 
Myanmar Myanmar
Angola Angola 
Vietnam Vietnam 
Madagascar Madagascar 
Mauritania Mauritania 
Colombia Colombia 
New Zealand New Zealand 
Belize Belize 
Sri Lanka Sri Lanka 
Lebanon Lebanon 
Venezuela Venezuela 
Hong Kong Hong Kong 
Belarus Belarus 
Israel Israel 
Tanzania Tanzania 
Congo (Kinshasa) Congo (Kinshasa)
Botswana B

In [4]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# print happy_data
happyX = []
happyY = []

for i in happy_data:
    happyX.append(i[0])
    happyY.append(i[2])
    
happyX = np.array(happyX)
happyY = np.array(happyY)
print happyX.shape
print happyY.shape

def mse_baseline(y_train, y_test):
    avg = np.average(y_train)
    print avg
    y_predict = np.full(y_test.shape, avg)
    print "baseline", mean_squared_error(y_test, y_predict)

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(happyX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = happyX[train_index], happyX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    mse_baseline(y_train, y_test)

(151,)
(151,)
5.3114299965
baseline 1.06183343465
5.43248514136
baseline 1.41324390412
5.35738614999
baseline 1.40885188022


In [50]:
import math
allArr = np.zeros(19)
for i in country_data:
    for j in range(2,19):
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                allArr[j] += i[j]
        else:
            allArr[j] +=(float(i[j].replace(',','.')))
means = allArr/float(len(country_data))

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

countriesX = [] 

# train using one column from 
# for i in country_data:
#     #comment this line out if not float data
#     f = float(i[7].replace(',','.'))
#     countriesX.append([f])
    
for i in country_data:
    info = []
    for j in range(2,19): 
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                info.append(i[j])
            else:
                info.append(means[j])
        else:
            info.append(float(i[j].replace(',','.')))  
    countriesX.append(info)
    
    
countriesX = np.array(countriesX)
print countriesX.shape
print happyY.shape
# lab_enc = preprocessing.LabelEncoder()
# happyY = lab_enc.fit_transform(happyY)
    
def mse_clf(X_train, X_test, y_train, y_test):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_predict = lr.predict(X_test)
    print "Linear Regression", mean_squared_error(y_test, y_predict)
    
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(countriesX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = countriesX[train_index], countriesX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    mse_clf(X_train, X_test, y_train, y_test)

(151, 1)
(151,)
Linear Regression 0.56722128632
Linear Regression 0.74338643276


/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [16]:
from sklearn.cluster import KMeans
from sklearn import preprocessing

countriesX = [] 

# train using one column from 
for i in country_data:
    #comment this line out if not float data
    f = float(i[7].replace(',','.'))
    countriesX.append([f])
    
countriesX = np.array(countriesX)
print countriesX.shape
print happyY.shape
# lab_enc = preprocessing.LabelEncoder()
# happyY = lab_enc.fit_transform(happyY)
    
def mse_clf(X_train, X_test, y_train, y_test):
    clusters=10
    km = KMeans(n_clusters=clusters)
    km.fit(X_train)
    test_clusters = km.predict(X_test)
    sums = np.zeros(clusters)
    counts = np.zeros(clusters)
    averages = np.zeros(clusters)
    
    for i, label in enumerate(km.labels_):
        sums[label] += y_train[i]
        counts[label] += 1
    
    for i in range(clusters):
        averages[i] = sums[i]/counts[i]
    
    y_predict = np.zeros(len(y_test))
    for i in range(len(test_clusters)):
        y_predict[i] = averages[test_clusters[i]]
    
    print "Clustering + Average", mean_squared_error(y_test, y_predict)
    
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(countriesX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = countriesX[train_index], countriesX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    mse_clf(X_train, X_test, y_train, y_test)

(151, 1)
(151,)
Clustering + Average 0.601278880274
Clustering + Average 0.524815619687
Clustering + Average 0.587169706779
